#### Importamos librerías con las que trabajaremos

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

#### Descargamos fichero shp Región Metropolitana para definir poligono de interés para POIs

In [5]:
rm_comunas = gpd.read_file('../GeoPython/Chile/Censo/R13/COMUNA_C17.shp')

In [6]:
# proyectamos coordenadas
rm_comunas.to_crs(4326, inplace=True)

In [7]:
# seleccionamos las comunas de interés

comunas_sel = rm_comunas.loc[~rm_comunas['NOM_COMUNA'].isin(['TILTIL','LAMPA', 'MARÍA PINTO', 'CURACAVÍ','CALERA DE TANGO',
                                                             'SAN JOSÉ DE MAIPO', 'PIRQUE','EL MONTE','TALAGANTE', 'SAN PEDRO', 'ALHUÉ',
                                                             'PAINE', 'BUIN','ISLA DE MAIPO','PEÑAFLOR', 'PADRE HURTADO','MELIPILLA'])].copy()

In [8]:
# con la función dissolve convertimos en un solo gran polígono
comunas_aoi = comunas_sel.dissolve()

In [3]:
# tags de OpenStreetMap
tags_restaurants = {'amenity':["bar", "cafe", "fast_food", "pub", "restaurant", "food_court"]}
tags_shop = {"shop": ["department_store", "supermarket", "mall", "hairdresser", "beauty", "furniture"]}
tags_cinema = {'amenity':['cinema', 'theatre']}
tags_banks = {'amenity':"bank"} 
tags_hotel = {'tourism':'hotel'}
tags_airport = {'aeroway':'aerodrome'}
tags_trains = {'public_transport':'station'}
tags_parking = {'amenity':'parking'}
tags_hospitals = {'amenity':['clinic','hospital']}
tags_ev = {'amenity':'charging_station'}

#### Obtenemos POIs de OSM

In [9]:
pois_restaurants = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_restaurants)

In [17]:
pois_shop = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_shop)

In [18]:
pois_cinema = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_cinema)

In [19]:
pois_banks = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_banks)

In [20]:
pois_hotel = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_hotel)

In [21]:
pois_airport = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_airport)

In [22]:
pois_trains = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_trains)

In [23]:
pois_parking = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_parking)

In [24]:
pois_hospitals = ox.geometries.geometries_from_polygon(polygon = comunas_aoi.iloc[0]['geometry'], tags=tags_hospitals)

In [26]:
# Obtenemos coordenadas Lon/Lat a partir de centroides
warnings.filterwarnings('ignore')

pois_restaurants['lat'] = pois_restaurants.centroid.y
pois_restaurants['lon'] = pois_restaurants.centroid.x

pois_shop['lat'] = pois_shop.centroid.y
pois_shop['lon'] = pois_shop.centroid.x

pois_cinema['lat'] = pois_cinema.centroid.y
pois_cinema['lon'] = pois_cinema.centroid.x

pois_banks['lat'] = pois_banks.centroid.y
pois_banks['lon'] = pois_banks.centroid.x

pois_hotel['lat'] = pois_hotel.centroid.y
pois_hotel['lon'] = pois_hotel.centroid.x

pois_airport['lat'] = pois_airport.centroid.y
pois_airport['lon'] = pois_airport.centroid.x

pois_trains['lat'] = pois_trains.centroid.y
pois_trains['lon'] = pois_trains.centroid.x

pois_parking['lat'] = pois_parking.centroid.y
pois_parking['lon'] = pois_parking.centroid.x

pois_hospitals['lat'] = pois_hospitals.centroid.y
pois_hospitals['lon'] = pois_hospitals.centroid.x

#### Transformamos a DF más acotados

In [28]:
df_rest = pois_restaurants.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_shop = pois_shop.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_cinema = pois_cinema.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_banks = pois_banks.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_hotel = pois_hotel.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_airport = pois_airport.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_trains = pois_trains.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_parking = pois_parking.reset_index()[['osmid', 'name', 'lat', 'lon']]
df_hospitals = pois_hospitals.reset_index()[['osmid', 'name', 'lat', 'lon']]

#### Unimos todos los POIs y creamos nuevo DF

In [31]:
locations = pd.concat([df_rest, df_shop, df_cinema, df_banks, df_hotel, df_airport, df_trains, df_parking, df_hospitals], ignore_index=True)

In [32]:
# chequeamos el join
len(locations), len(df_rest) + len(df_shop) + len(df_cinema) + len(df_banks) + len(df_hotel) + len(df_airport) + len(df_trains)+len(df_parking) + len(df_hospitals)

(9973, 9973)

In [33]:
# convertimos a fichero csv
locations.to_csv('../GeoPython/Chile/EV_Chargers/POIs_location.csv')